# Sachetan B Heralagi

**Email:** manuheralagi4@gmail.com  
**Phone:** 8431367857

## Architecture and Data Explanation

### Data
- The code loads data from `output1.csv`, which represents game states of a 3x3 grid game (like Tic-Tac-Toe).
- Each row in the CSV represents a game, with columns `MOVE1` to `MOVE7` indicating player moves.
- The `CLAS` column denotes the game outcome (win, loss, draw).
- The goal is to train a model to predict the game outcome based on the board state.

### Node Features
- For each game, the board state is converted into a 9x1 tensor, where each element represents a cell on the board.
- The value is `1` for player 1's move, `-1` for player 2's move, and `0` for an empty cell.

### Edge Index (Adjacency Matrix)
- The `edge_index` tensor defines the connections between cells on the board.
- It's a 2x16 tensor, where each column represents an edge connecting two nodes (cells).
- The values in each column are the indices of the connected nodes.
- In this case, it defines an 8-connected neighborhood, meaning each cell is connected to its immediate neighbors (horizontal, vertical, and diagonal).

### Labels
- The `CLAS` column is converted to numerical labels using the `class_mapping` dictionary.
- 'wi' (win) is mapped to `1`, 'los' (loss) to `0`, and 'dra' (draw) to `2`.

### PyTorch Geometric Data
- The node features, edge index, and labels are combined into PyTorch Geometric `Data` objects.
- These objects represent graph structures, where nodes are cells on the board, edges represent connections, and labels indicate the game outcome.

### Data Loader
- The `DataLoader` is used to batch the data for efficient training.

## Model Architecture (Implied)
- Imports `GCNConv` from `torch_geometric.nn`, i.e., using Graph Convolutional Network (GCN).
- GCNs are neural networks designed to operate on graph-structured data.
- They learn to represent nodes by aggregating information from their neighbors, effectively capturing the relationships between cells on the board.

## Results
- In this case, we don't focus on overfitting due to the limited number of possible board states, which are nearly 200,000 unique states.
- The model was trained for 1250 epochs, achieving an accuracy of approximately 72%, which can be used to play the game.

## Summary
The code prepares graph-structured data representing game states and sets up the foundation for training a GCN model to predict game outcomes based on board configurations.


In [5]:
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader



In [2]:

# Load the dataset
df = pd.read_csv('output1.csv')
print(df.shape)

# Define mappings
class_mapping = {'wi': 1, 'los': 0, 'dra': 2}

# Prepare lists for node features, edge indices, and labels
node_features = []
edge_indices = []
labels = []

# Define adjacency matrix for a 3x3 grid (8-connected neighborhood for simplicity)
edge_index = torch.tensor([
    [0, 1], [1, 2], [3, 4], [4, 5], [6, 7], [7, 8],
    [0, 3], [3, 6], [1, 4], [4, 7], [2, 5], [5, 8],
    [0, 4], [4, 8], [2, 4], [4, 6]
], dtype=torch.long).t().contiguous()

for index, row in df.iterrows():
    # Initialize board state
    board = [0] * 9
    
    # Apply moves up to the point where the game ends (first occurrence of ?)
    for i, move in enumerate([row[f'MOVE{i+1}'] for i in range(7)]):
        if move == '?':
            break
        move = int(move)  # Convert move to integer
        board[move] = 1 if i % 2 == 0 else -1  # Player 1's move is 1, Player 2's move is -1
    
    # Convert board state to features (shape [9, 1])
    features = torch.tensor(board, dtype=torch.float).view(-1, 1)
    
    # Convert class to label
    class_label = row['CLAS']
    if class_label not in class_mapping:
        print(f"Warning: Class {class_label} not in mapping.")
    y = torch.tensor(class_mapping.get(class_label, -1), dtype=torch.long)  # Default to -1 if not found
    
    # Append features, edge indices, and labels for the board state
    node_features.append(features)
    edge_indices.append(edge_index)
    labels.append(y)

# Create PyG Data objects
data_list = [Data(x=x, edge_index=edge_index, y=y) for x, edge_index, y in zip(node_features, edge_indices, labels)]

# Create a dataset and data loader
dataset = data_list
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

print(f'Dataset size: {len(dataset)}')
print(f'Sample Data: {data_loader.dataset[0]}')


(2436, 8)
Dataset size: 2436
Sample Data: Data(x=[9, 1], edge_index=[2, 16], y=0)


/home/sachetan/e/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)  # Output dimension should match the number of classes
    
    def forward(self, x, edge_index, batch):
        # Ensure x has the correct shape (num_nodes, input_dim)
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = global_mean_pool(x, batch)  # Aggregate node features into graph-level features
        x = self.fc(x)
        return x

In [7]:
from torch_geometric.nn import GCNConv, global_mean_pool

# Create model, criterion, and optimizer
model = GCN(input_dim=1, hidden_dim=16, output_dim=3)  # Ensure input_dim matches your feature dimension
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs =1250
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data in data_loader:
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data.x, data.edge_index, data.batch)
        
        # Ensure that output and target have compatible shapes
         # Should be [batch_size]
        
        # Calculate loss
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(output, dim=1)
        correct += (predicted == data.y).sum().item()
        total += data.y.size(0)
    
    accuracy = correct / total if total > 0 else 0
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader):.4f}, Accuracy: {accuracy:.4f}')

print("Training complete.")


/home/sachetan/e/lib/python3.12/site-packages/torch/autograd/graph.py:768: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/1250, Loss: 1.0276, Accuracy: 0.5049
Epoch 2/1250, Loss: 0.9976, Accuracy: 0.5460
Epoch 3/1250, Loss: 0.9829, Accuracy: 0.5616
Epoch 4/1250, Loss: 0.9628, Accuracy: 0.5583
Epoch 5/1250, Loss: 0.9551, Accuracy: 0.5776
Epoch 6/1250, Loss: 0.9293, Accuracy: 0.5759
Epoch 7/1250, Loss: 0.8990, Accuracy: 0.5895
Epoch 8/1250, Loss: 0.8800, Accuracy: 0.5800
Epoch 9/1250, Loss: 0.8718, Accuracy: 0.5956
Epoch 10/1250, Loss: 0.8561, Accuracy: 0.6026
Epoch 11/1250, Loss: 0.8504, Accuracy: 0.6108
Epoch 12/1250, Loss: 0.8441, Accuracy: 0.6059
Epoch 13/1250, Loss: 0.8417, Accuracy: 0.6022
Epoch 14/1250, Loss: 0.8380, Accuracy: 0.6112
Epoch 15/1250, Loss: 0.8268, Accuracy: 0.6178
Epoch 16/1250, Loss: 0.8252, Accuracy: 0.6199
Epoch 17/1250, Loss: 0.8396, Accuracy: 0.6117
Epoch 18/1250, Loss: 0.8214, Accuracy: 0.6166
Epoch 19/1250, Loss: 0.8258, Accuracy: 0.6252
Epoch 20/1250, Loss: 0.8350, Accuracy: 0.5956
Epoch 21/1250, Loss: 0.8201, Accuracy: 0.6195
Epoch 22/1250, Loss: 0.8137, Accuracy: 0.62

In [6]:
import torch

# Saving the trained model
torch.save(model.state_dict(), 'path_to_saved_model.pth')
